In [74]:
# 将inspire触觉手合并到adam_standard上

import xml.etree.ElementTree as ET
from dataclasses import dataclass
import os
import shutil
import glob


@dataclass
class Cfg:
    name: str
    prefix: str
    postfix: str
    wrist_roll_urdf_path: str
    wrist_roll_prior: str
    wrist_roll_name: str
    wrist_roll_parent: str
    wrist_roll_child: str
    inspire_hand_urdf_path: str


tree = ET.parse("adam_standard.urdf")
root = tree.getroot()


def merge_hand(hand_cfg: Cfg):
    wrist_link_tree = ET.parse(hand_cfg.wrist_roll_urdf_path)
    wrist_link_root = wrist_link_tree.getroot()

    idx = 0
    for elem in root:
        idx += 1
        if elem.tag == "joint" and elem.attrib["name"] == hand_cfg.wrist_roll_prior:
            # print(elem.tag, elem.attrib)
            break
    for elem in wrist_link_root:
        if elem.tag == "link":
            root.insert(idx, elem)

    idx += 1
    wrist_roll = ET.Element("joint", name=hand_cfg.wrist_roll_name, type="revolute")
    ET.SubElement(wrist_roll, "origin", xyz="0 0 0", rpy="0 0 0")
    ET.SubElement(wrist_roll, "parent", link=hand_cfg.wrist_roll_parent)
    ET.SubElement(wrist_roll, "child", link=hand_cfg.wrist_roll_child)
    ET.SubElement(wrist_roll, "axis", xyz="1 0 0")
    ET.SubElement(
        wrist_roll,
        "limit",
        effort="6.4",
        lower="-0.907",
        upper="0.907",
        velocity="4.9218",
    )
    root.insert(idx, wrist_roll)
    idx += 1

    inspire_force_tree = ET.parse(hand_cfg.inspire_hand_urdf_path)
    inspire_force_root = inspire_force_tree.getroot()
    for elem in inspire_force_root:
        if elem.tag == "link" and elem.attrib["name"] == "base_link":
            base_link_idx = idx + 1
        if elem.tag in ["link", "joint"]:
            elem.attrib["name"] = f'{hand_cfg.prefix}{elem.attrib["name"]}'

            for sub_elem in elem.iter():
                for key, value in sub_elem.attrib.items():
                    if ".stl" in value.lower():
                        sub_elem.attrib[key] = value.replace(
                            ".STL", f"{hand_cfg.postfix}.STL"
                        )
                if sub_elem.tag in ["parent", "child"]:
                    sub_elem.attrib["link"] = (
                        f"{hand_cfg.prefix}{sub_elem.attrib['link']}"
                    )
                if sub_elem.tag == "mimic":
                    sub_elem.attrib["joint"] = (
                        f"{hand_cfg.prefix}{sub_elem.attrib['joint']}"
                    )
            root.insert(idx, elem)
            idx += 1

    base_link_joint = ET.Element(
        "joint", name=f"inspire_{hand_cfg.name}_base_link_joint", type="fixed"
    )
    ET.SubElement(base_link_joint, "origin", xyz="0 0 0", rpy="0 0 0")
    ET.SubElement(base_link_joint, "parent", link=hand_cfg.wrist_roll_child)
    ET.SubElement(base_link_joint, "child", link=f"{hand_cfg.prefix}base_link")
    ET.SubElement(base_link_joint, "axis", xyz="0 0 0")
    root.insert(base_link_idx, base_link_joint)


left_hand_cfg = Cfg(
    name="left_hand",
    prefix="l_",
    postfix="_l",
    wrist_roll_urdf_path="../res/wristRollLeft/urdf/wristRollLeft.urdf",
    wrist_roll_prior="wristPitch_Left",
    wrist_roll_name="wristRoll_Left",
    wrist_roll_parent="wristPitchLeft",
    wrist_roll_child="wristRollLeft",
    inspire_hand_urdf_path="../third-party/inspire/urdf_left_with_force_sensor_no_fl/urdf/urdf_left_with_force_sensor.urdf",
)
right_hand_cfg = Cfg(
    name="right_hand",
    prefix="r_",
    postfix="_r",
    wrist_roll_urdf_path="../res/wristRollRight/urdf/wristRollRight.urdf",
    wrist_roll_prior="wristPitch_Right",
    wrist_roll_name="wristRoll_Right",
    wrist_roll_parent="wristPitchRight",
    wrist_roll_child="wristRollRight",
    inspire_hand_urdf_path="../third-party/inspire/urdf_right_with_force_sensor_no_fl/urdf/urdf_right_with_force_sensor.urdf",
)
merge_hand(left_hand_cfg)
merge_hand(right_hand_cfg)


def replace_str_in_elem(old_str, new_str):
    for elem in root.iter():
        if elem.text and old_str in elem.text:
            elem.text = elem.text.replace(old_str, new_str)
        if elem.tail and old_str in elem.tail:
            elem.tail = elem.tail.replace(old_str, new_str)
        for key, value in elem.attrib.items():
            if old_str in value:
                elem.attrib[key] = value.replace(old_str, new_str)


replace_str_in_elem("package://wristRollLeft/", "")
replace_str_in_elem("package://wristRollRight/", "")
replace_str_in_elem("package://urdf_left_with_force_sensor/", "")
replace_str_in_elem("package://urdf_right_with_force_sensor/", "")

new_folder = "../adam_standard_inspire_force/"
if os.path.exists(new_folder):
    shutil.rmtree(new_folder)
os.makedirs(new_folder, exist_ok=True)
ET.indent(tree, space="\t")
root.attrib["name"] = "adam_standard_inspire_force"
tree.write(
    f"{new_folder}adam_standard_inspire_force.urdf",
    encoding="utf-8",
    xml_declaration=True,
)


####### meshes #######
new_meshs_folder = os.path.join(new_folder, "meshes")
shutil.copytree("meshes", new_meshs_folder)
shutil.copytree("../res/wristRollLeft/meshes", new_meshs_folder, dirs_exist_ok=True)
shutil.copytree("../res/wristRollRight/meshes", new_meshs_folder, dirs_exist_ok=True)


def move_and_rename_stl_files(src_dirs, dst_dir, postfix):
    os.makedirs(dst_dir, exist_ok=True)
    for src_dir in src_dirs:
        for stl_path in glob.glob(os.path.join(src_dir, "**", "*.STL"), recursive=True):
            base = os.path.basename(stl_path)
            name, ext = os.path.splitext(base)
            new_name = f"{name}{postfix}{ext}"
            dst_path = os.path.join(dst_dir, new_name)
            shutil.copy2(stl_path, dst_path)
            # print(f"Copied {stl_path} -> {dst_path}")


left_mesh_src = "../third-party/inspire/urdf_left_with_force_sensor_no_fl/meshes"
right_mesh_src = "../third-party/inspire/urdf_right_with_force_sensor_no_fl/meshes"
move_and_rename_stl_files([left_mesh_src], new_meshs_folder, "_l")
move_and_rename_stl_files([right_mesh_src], new_meshs_folder, "_r")
